This shows an example of using Bing Image Creator which does not have an API

You need to store your cookie in .env
TODO: add docs here

In [1]:
import os
import urllib.parse

from pyrit.models import PromptTemplate
from pyrit.orchestrator import PromptSendingOrchestrator
from pyrit.prompt_target import HTTP_Target

from pyrit.common import default_values


default_values.load_default_env()

from pyrit.common import default_values
from pyrit.models import PromptRequestPiece
from pyrit.models import PromptRequestPiece

default_values.load_default_env()

http_resp = f"""
Host: www.bing.com
Cookie: {os.getenv("BING_COOKIE")}
Content-Length: 21
Cache-Control: max-age=0
Sec-Ch-Ua: "Chromium";v="127", "Not)A;Brand";v="99"
Sec-Ch-Ua-Mobile: ?0
Sec-Ch-Ua-Full-Version: ""
Sec-Ch-Ua-Arch: ""
Sec-Ch-Ua-Platform: "Windows"
Sec-Ch-Ua-Platform-Version: ""
Sec-Ch-Ua-Model: ""
Sec-Ch-Ua-Bitness: ""
Sec-Ch-Ua-Full-Version-List: 
Accept-Language: en-US
Upgrade-Insecure-Requests: 1
Origin: https://www.bing.com
Content-Type: application/x-www-form-urlencoded
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.6533.100 Safari/537.36
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7
Sec-Fetch-Site: same-origin
Sec-Fetch-Mode: navigate
Sec-Fetch-User: ?1
Sec-Fetch-Dest: document
Referer: https://www.bing.com/images/create?FORM=GENILP
Accept-Encoding: gzip, deflate, br
Priority: u=0, i
"""

## Add the prompt you want to send to the URL
prompt = "roakey racoon"
prompt_url_safe = urllib.parse.quote(prompt)
url = f"https://www.bing.com/images/create?q={prompt_url_safe}&rt=3&FORM=GENCRE"

# Add the prompt to the body of the request
body_query = prompt.replace(" ", "+")
body = f"q={body_query}&qs=ds"
with HTTP_Target(http_request=http_resp, url=url, body=body) as target_llm:
    request = PromptRequestPiece(
        role="user",
        original_value=prompt,
    ).to_prompt_request_response()

    resp = await target_llm.send_prompt_async(prompt_request=request)  # type: ignore
    print(resp)

None: assistant: b'[\xdf\x1723hV\x8b{\xb8CD\xd2Y=\x04Tgb\x8c\x1b\xfa\xe3\xd7\x9f\x7f\xff!08&\xe0\xc3\xb4l\xc7\xf5|~\xff\x99\xaf\xfe\xbb\x7fm\x17{\nm\x19 :\xa2(\xff\x92\x10>\xd1\xd8\x8e\xe3\xf8\xe6\xf7\xed(\xba\x1a\x88<\x10\x11S\x80.>\x925\x12g\xba\xd9v\xbf\xed\xb4V\xf5\xfd\xe9J\xa4Q\xaf\xd2i\xa4\x01\x97ty\xfb^5\xb3w\x10\x8b\xa8\xbcB\xa0\x81\xcc\xb4\x9e\xe1\xc4\xdcN\xff|\xf8\x93M\xad\xd7\x97\xdd+\xcd\xf6\t\x90[r\r\x1bL\x94 \xb9\x80\x1b\x1fF ,D?\x8a)\x99\xfco\xbe\xda\xff7\x7f\xdd=\x98S\xbbn\xd1\xd8\xa5mZ\xb2\x9dO\x91\x81\xb5l\'q\\\xe58\xaa\xd8\xa9\xcfQ\xe9j\xf1\x03\x92\x88(\x80\x016$\xeb\x88Z\xab\'\x83\x1e\x0cF\xfd\xff\xfb\xa6V9DI\xb3\xc6\x99\xc8\x05\xb9\x886c\x9c\x89\xd6\x84\x1b^\xf3\x82\xf7\xff\xaf\xe0\x97\t\xaa\x00\x04U\x00\x82\x02\xc0\xa0P@P y\xdf{\xbf\xc0\xff\x7f\x15\xa9\xaa\x02\xa9.\x00R\x0f\x00\xb2\xe7\x80 {N\x01\xa4\xe6\x90\x94\xd6\x16H\x04\xa0\xd8\x01)\xf59#\xad&\xb0\'\x9ft\rHi\x9c\xa49G=\xd6E\xbb\xd9\x06\xa1zmk\x8d\x8b&\\\x97\x04\x1b\x04\xe1\xc6\xf1>\x86i\xf7\xb4\xddv\xdd\xee\x

In [2]:
http_prompt_target = HTTP_Target(http_request=http_resp, url=url, body=body)

with PromptSendingOrchestrator(prompt_target=http_prompt_target) as orchestrator:
    response = await orchestrator.send_prompts_async(prompt_list=[prompt])  # type: ignore
    print(response[0])

HERE IN ORCHESTRATOR, ['roakey racoon']
{'__type__': 'HTTP_Target', '__module__': 'pyrit.prompt_target.http_target'}: assistant: b'[\xd2\x92Rv@ad\x1e\x00\x00j\xfd{\xb8KDV:=\x04\xa8\xaaeB\xbc\xbdj@\xfc\xf2\xdb\x1f\x7f\xfd\xf3\xdf\x7f\t\x0c\x8e\t\xfc\x87i\xd9\x8e\xeb\xf1\xfa\xfc\xfe3_\r\xef\xfd\x1c\xee\x14\xda2@\xf8DQv\xe2$\x84\x9f5\xb6\xe38\xea\xe6\xd7\xd8\xfbU\xb4\x1c\x88|\x14\x11S\x80\x16\x1f\xc9\x1a\x893\xbd\\{\xbfvZZ\xf9\x7f\xbaR\xd35\x9e\x91-a@B\xb1\xbc\x0eq_H\xbb\x8e/PA \xf1\x06\x01\x0bt\xb7\xc6*\xfd\xfa\xb7\xd3?\x1f\xfedS\xa9\xd7\x97Yo\xbd\xb3}\x02\xe4\x96\\#\x06\x13%H.\xe0\xc6\x87\x11\x08\x840m)\xa6d|\xef\xef\xb5\xbe\xaf_\xbb\xee\x1c\xac\xbaTh\x93t\xf7\xccB*>yO\xa6_\xde\xb6\xef\x930>\x18\x84\xd1\x98H\xac$\xecx\r\xff\xfb6\xeb\xff\xff\xdf\xf0\xf1otn\\R\xb316d\xc4\xa5\xb0\xc8P)\xba\x92T:C\xf5@qX\xb2-\xdb*\x8c\xe4H[\x10\x1a\xb8~\xf6\xae\xf2?\xf0\xf1_r\x00\xebmWu5);\x1b\xf0\xc1D:B\x07=\x1e\xdfz\x04d\xaf\x03~\xafjU\x94\xfe\xd1\x1a\xada;I\xad\x1eo\xda\xf95\xfd\x8a`\x91\xc4\x12\xac\xc2\